# LSTM

In [ ]:
# https://wikidocs.net/44249

In [25]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re                              # 정규표현식을 지원한다 
from konlpy.tag import Okt             # 한국어 처리 패키지 
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer  # 토큰화(나눠준다고생각하자)
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences # 샘플의 길이 동일하게(패딩), 길이가 다른 경우 0을 넣어서 맞춰준다


## 데이터 확인, train,test 나누기

In [26]:
data =pd.read_excel("C:/Users/ICT01_17/Documents/project2/wordtrainfinal.xlsx")

In [27]:
# 행 무작위로 순서바꾸자.
data=data.sample(frac=1)  # >> 모든행을 임의의 순서로 반환한다.
data

,id,document,label
1393,1395,우기다,0
126,586,두렵고 싫었다,4
830,832,간절하다,3
956,958,분하다,0
1333,1335,벅차다,1
...,...,...,...
663,43,화내면,0
449,78,이쁜 선물 받았습니다,1
1453,1455,짜증나다,0
608,231,피곤,3


In [28]:
test_data=data.iloc[:200,:]

In [29]:
train_data = data.iloc[200:,:]

## 데이터 전처리

### 특수문자 제거 (한글만 남기고 제거하기위해서 정규 표현식을 사용)

In [30]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# >> 한글과 공백을 제외하고 모두 제거한다는 뜻

C:\Users\ICT01_17\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### 토큰화와 불용어(조사나 접속사 같은 것) 제거

In [31]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','을',
             '를','으로','자','에','와','한','하다','로','이다']

# >> 토큰화를 위한 형태소 분석기는 KoNLPy 의 Okt 를 사용한다
# >> KoNLPy : 띄어쓰기,알고리즘 정규화를 이용해서 맞춤법 틀린 문장 어느정도 고쳐주면서 형태소 분석과 품사를 태깅

okt = Okt() # KoNLPy 에서 제공하는 형태소 분석기이다.(영어는 띄어쓰기 기준으로 토큰화하지만 한국어는 주로 형태소로 나눈다)

In [32]:
X_train = []
for sentence in train_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True, norm=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [33]:
#print(X_train) # >>형태소 토큰화가 진행되었다. 

In [34]:
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
X_test=[]
for sentence in test_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True, norm=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

C:\Users\ICT01_17\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
#X_test

## 정수 인코딩(X_train, X_test 에 대해서)

In [36]:
max_words = 2000
tokenizer = Tokenizer(num_words=max_words) # 상위 2000개 단어만 보존

tokenizer.fit_on_texts(X_train) # 단어 인덱스를 구축 

X_train = tokenizer.texts_to_sequences(X_train) # 문자열을 정수 인덱스의 리스트로 변환
X_test = tokenizer.texts_to_sequences(X_test)
#print(X_train) # 단어 대신 단어에 대한 인덱스 부여 

In [37]:
#print(X_train)

## 데이터의 길이 맞추기

In [38]:
print('글자 최대 길이 :',max(len(l) for l in X_train))

# 모델이 처리할 수 있도록 X_train, X_test 의 모든 샘플의 길이를 동일하게 하자

글자 최대 길이 : 10


In [39]:
max_len =8 # 길이를 10으로 정했다
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

print(X_train) # 패딩 된것을 확인할수있다( 없는 부분은 0으로 채움)

[[  0   0   0 ...   0  36  36]
 [  0   0   0 ...   0   0 365]
 [  0   0   0 ...   0 176 177]
 ...
 [  0   0   0 ...   0   0  20]
 [  0   0   0 ...   0   0 164]
 [  0   0   0 ...   0  34 152]]


In [40]:
X_train.shape

(1299, 8)

## 레이블 저장

In [41]:
y_train = np.array(train_data['label'])
# print(y_train)
y_test = np.array(test_data['label'])

## LSTM로 감성 분류

In [42]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [43]:
#model = Sequential()

#model.add(Embedding(max_words,100))  # 임베딩 층(이것도 인공 신경망의 층 중 하나) 
                        #embedding(number of samples, input_length) >> 벡터의 차원은 100으로

#model.add(LSTM(128)) # 감정 분류를 위해서 LSTM 사용

#model.add(Dense(8, activation='relu'))
#model.add(Dense(1, activation='relu'))
#model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
#history = model.fit(X_train, y_train, epochs=10, batch_size=20, validation_split=0.2)

In [ ]:
#model = Sequential()
#model.add(Embedding(max_words,100))  
#model.add(LSTM(128)) 
#model.add(Dense(8, activation='relu'))
#model.add(Dense(1, activation='relu'))
#model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
#history = model.fit(X_train, y_train, epochs=10, batch_size=20, validation_split=0.2)

In [44]:
#print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))
#print(model.evaluate(X_test,y_test))

In [ ]:
# 정규화(scaling)
#from sklearn.preprocessing import MinMaxScaler

#sc=MinMaxScaler()
#train_sc = sc.fit_transform(X_train)
#test_sc = sc.transform(X_test)

In [45]:
model3 =Sequential()
model3.add(Embedding(max_words,30))
model3.add(LSTM(40))
model3.add(Dense(1, activation='relu'))
model3.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy']) 
model3.fit(X_train, y_train, epochs=50, batch_size=30, verbose=1,validation_split=0.2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 1039 samples, validate on 260 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
1039/1039 [==============================] - 1s 550us/sample - loss: 4.6325 - acc: 0.2570 - val_loss: 2.2251 - val_acc: 0.3269
Epoch 2/50
1039/1039 [==============================] - 0s 167us/sample - loss: 2.3988 - acc: 0.2685 - val_loss: 2.1077 - val_acc: 0.3269
Epoch 3/50
1039/1039 [==============================] - 0s 157us/sample - loss: 2.2554 - acc: 0.2685 - val_loss: 2.0547 - val_acc: 0.3269
Epoch 4/50
1039/1039 [==============================] - 0s 161us/sample - loss: 2.0387 - acc: 0.2685 - val_loss: 1.9750 - val_acc: 0.3269
Epoch 5/50
1039/1039 [==============================] - 0s 165us/sample - loss: 1.7440 - acc: 0.2705 - val_loss: 1.8230 - val_acc: 0.3269
Epoch 6/50
1039/1039 [=============================

In [46]:
print(model3.evaluate(X_test, y_test))

200/200 [==============================] - 0s 50us/sample - loss: 2.8339 - acc: 0.2350
[2.8339485549926757, 0.235]


## 모델저장

In [ ]:
#TRAINED_CLASSIFIER_PATH = "dual_encoder_lstm_classifier.h5" # 모델이름

#model3.save(TRAINED_CLASSIFIER_PATH)

In [47]:
from keras.models import load_model
model3.save('lstm.h5')

## 예측

In [48]:
#1단계 : 기분을 대입
feel = '아침엔 악몽을 꾸어서 짜증났지만 오늘 문제도 잘 풀리고 컨디션이 좋아져서 상쾌한 오전이다.'

In [49]:
#2단계 : 대입한 기분 전처리( 토큰화, 벡터화 등) 
def Preprocess(feel):
    X = okt.morphs(feel, stem=True, norm=True)
    X = [word for word in X if not word in stopwords]
    
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=max_len)
    return X      # X는 예측할 데이터 전처리한 결과    

In [50]:
#3단계 : 모델에 대입 
predict = model3.predict(Preprocess(feel))   # 전처리한 결과 모델에 넣었다
# print(predict)  # 예측결과

In [1]:
#predict

In [52]:
# https://codepractice.tistory.com/71
def Delete(predict):
    predict = predict.reshape(-1).astype('int')
    predict = np.around(predict)
    from collections import Counter    # 최빈값
    list = []
    for i in range(len(predict)):
        if predict[i]!=2 and predict[i]!=5:   # 2, 5제거(중성단어)
            a=list.append(predict[i])       
            
    if Counter(list).most_common(1)[0][0] ==0:print('아마 당신의 기분은 angry 로 추정됩니다')
    if Counter(list).most_common(1)[0][0] ==1:print('아마 당신의 기분은 happy 로 추정됩니다')
    if Counter(list).most_common(1)[0][0] ==3:print('아마 당신의 기분은 sad 로 추정됩니다')
    if Counter(list).most_common(1)[0][0] ==4:print('아마 당신의 기분은 afraid 로 추정됩니다')


In [53]:
Delete(predict)

아마 당신의 기분은 happy 로 추정됩니다


# LSTM 신경망의 딥러닝을 이용한 EEG 신호로부터 공포감정 인식시트템

In [ ]:
http://sclab.yonsei.ac.kr/publications/Papers/KC/2016_KCC_fall_WGH.pdf
https://wikidocs.net/49071

In [54]:
# 기존의 방식 (MLP) : 다층 퍼셉트론으로 FFNN 의 기본적 형태

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [61]:
X_train.shape

(1299, 8)

In [64]:
# 다층 퍼셉트론 모델(MLP)
num_classes = 5
def fit_and_evaluate(X_train,y_train, X_test, y_test):
    model4 = Sequential()
    model4.add(Dense(256, input_shape=(-1,-1,-1), activation='relu'))
    model4.add(Dropout(0.5))
    model4.add(Dense(128, activation='relu'))
    model4.add(Dropout(0.5))
    model4.add(Dense(num_classes, activation='softmax'))
    
    model4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model4.fit(X_train, y_train, batch_size=128, epochs=50, verbose=1, validation_split=0.1)
    score = model4.evaluate(X_test, y_test, batch_size=128, verbose=0)
    return score[1]

# 4개의 층으로 구성되어있다.

In [65]:
fit_and_evaluate(X_train, y_train, X_test, y_test)
# MLP 보다는 LSTM 이 더 높다.

ValueError: Error converting shape to a TensorShape: Dimension -1 must be >= 0.

In [58]:
model4 = Sequential()
model4.add(Dense(256, input_shape=(max_len,), activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(128, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(num_classes, activation='softmax'))
    
model4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.1)
score = model4.evaluate(X_test, y_test, batch_size=128, verbose=0)

Train on 1169 samples, validate on 130 samples
Epoch 1/5
1169/1169 [==============================] - 0s 143us/sample - loss: 73.6738 - acc: 0.2335 - val_loss: 41.3735 - val_acc: 0.3154
Epoch 2/5
1169/1169 [==============================] - 0s 20us/sample - loss: 54.6838 - acc: 0.2216 - val_loss: 12.6064 - val_acc: 0.3000
Epoch 3/5
1169/1169 [==============================] - 0s 21us/sample - loss: 37.0925 - acc: 0.2558 - val_loss: 15.6365 - val_acc: 0.2308
Epoch 4/5
1169/1169 [==============================] - 0s 22us/sample - loss: 27.9200 - acc: 0.2472 - val_loss: 8.1466 - val_acc: 0.1923
Epoch 5/5
1169/1169 [==============================] - 0s 22us/sample - loss: 20.3700 - acc: 0.2352 - val_loss: 4.0270 - val_acc: 0.1923


In [59]:
feel2 = '너가 아파서 마음이 아팠고 슬펐다'
predict2=model4.predict(Preprocess(feel2))
predict2

array([[0.4575142 , 0.19024019, 0.00161356, 0.16755472, 0.1830773 ],
       [0.20553158, 0.20169427, 0.18605295, 0.20567496, 0.20104623],
       [0.21563472, 0.20600607, 0.16412838, 0.20911218, 0.20511867],
       [0.20553158, 0.20169427, 0.18605295, 0.20567496, 0.20104623],
       [0.22687425, 0.21019775, 0.14164497, 0.21225345, 0.20902966]],
      dtype=float32)

In [60]:
Delete(predict2)  #>>> MLP 보다 LSTM 이 효과적이다.

아마 당신의 기분은 angry 로 추정됩니다


# 여러개의 LSTM 셀로 이루어진 layer 로 여러층을 쌓은 심층 RNN

In [1]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.
C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ICT01_17\Anaconda3\lib\site-packages\tensorflow\python\framewo

In [80]:
model5 = tensorflow.keras.Sequential()
model5.add(InputLayer(input_shape=(1299,8)))
model5.add(LSTM(20, return_sequences = True))
model5.add(LSTM(20, return_sequences = True))
model5.add(LSTM(20, return_sequences = True))
model5.add(LSTM(20, return_sequences = False))
model5.add(Dense(46))
model5.add(Activation('relu'))
model5.compile(loss = 'sparse_categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

TypeError: The added layer must be an instance of class Layer. Found: <keras.layers.core.Activation object at 0x000002170A8D1E88>

In [71]:
model3 =tensorflow.keras.Sequential()
model3.add(Embedding(max_words,30))
model3.add(LSTM(40))
model3.add(Dense(1))`
model3.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy']) 

model3.fit(X_train, y_train, epochs=50, batch_size=30, verbose=1,validation_split=0.2)

Train on 1039 samples, validate on 260 samples
Epoch 1/50
1039/1039 [==============================] - 1s 675us/sample - loss: 4.5630 - acc: 0.2570 - val_loss: 2.2076 - val_acc: 0.3269
Epoch 2/50
1039/1039 [==============================] - 0s 193us/sample - loss: 2.4133 - acc: 0.2685 - val_loss: 2.0990 - val_acc: 0.3269
Epoch 3/50
1039/1039 [==============================] - 0s 163us/sample - loss: 2.2575 - acc: 0.2685 - val_loss: 2.0480 - val_acc: 0.3269
Epoch 4/50
1039/1039 [==============================] - 0s 163us/sample - loss: 2.0950 - acc: 0.2685 - val_loss: 1.9578 - val_acc: 0.3269
Epoch 5/50
1039/1039 [==============================] - 0s 167us/sample - loss: 1.7392 - acc: 0.2685 - val_loss: 1.8160 - val_acc: 0.3269
Epoch 6/50
1039/1039 [==============================] - 0s 164us/sample - loss: 1.2660 - acc: 0.2724 - val_loss: 1.7188 - val_acc: 0.3269
Epoch 7/50
1039/1039 [==============================] - 0s 162us/sample - loss: 0.8628 - acc: 0.2849 - val_loss: 1.7389 - val